In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings

warnings.simplefilter("ignore")

In [3]:
# scrapy player picture from statmuse
response = requests.get("https://www.statmuse.com/nba/players")
htmlcontent = response.content
results_soup = BeautifulSoup(htmlcontent, "html.parser")
search_results = results_soup.find_all("lazy-img", {"class": "img"})

name_lst = []
img_lst = []

for result in search_results:
    name_lst.append(result.get("alt"))
    img_lst.append(result.get("src"))

img_df = pd.DataFrame(data=[name_lst, img_lst]).T
img_df.columns = ["player", "url"]

for row in img_df.iterrows():
    response = requests.get(row[1]["url"])
    if response.status_code == 200:
        save_name = "{}_{}".format(row[0], row[1]["player"])
        print(save_name)
        with open("nba_player/{}.jpg".format(save_name), "wb") as f:
            f.write(response.content)

### espn nba player (not working)

In [7]:
# espn_teams_url = "http://www.espn.com/nba/players"
# teams_response = requests.get(espn_teams_url)
# teams_soup = BeautifulSoup(teams_response.content, "html.parser")
# area_soup = teams_soup.find_all("div", {"id": "my-players-table"})
# small_logos = area_soup[0].find_all("ul", {"class": "small-logos"})

# href_lst = []

# for area in small_logos:
#     for a in area.find_all("a", href=True):
#         if a["href"].startswith("http"):
#             href_lst.append(a["href"])

### nba offical player (active)

In [5]:
# get full page source with selunium
from selenium import webdriver
driver_path = r'/usr/local/bin/chromedriver'
browser = webdriver.Chrome(executable_path=driver_path)

browser.get("https://www.nba.com/players")
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

player_contents = soup.find("table", class_="players-list").find_all("td", class_="primary text RosterRow_primaryCol__19xPQ")

In [8]:
nba_host = "https://www.nba.com"
name_lst = []
url_lst = []
exp_lst = []

In [13]:
# scrapy player and url
for ind, content in enumerate(player_contents):
    try:
        player_url = content.find("a", href=True).get("href")
        player_res = requests.get(nba_host + player_url, verify=False)

        player_soup = BeautifulSoup(player_res.content, "html.parser")
        player_image = player_soup.find("img", class_="PlayerImage_image__1smob w-10/12 mx-auto mt-16 md:mt-24")

        name = player_image.get("alt")
        imgurl = player_image.get("src")

        name_lst.append(name)
        url_lst.append(imgurl)

        print(ind, name)
    except:
        exp_lst.append(player_url)

for ind, exp_player in enumerate(exp_lst):
    player_res = requests.get(nba_host + exp_player, verify=False)
    
    player_soup = BeautifulSoup(player_res.content, "html.parser")
    player_image = player_soup.find("img", class_="PlayerImage_image__1smob w-10/12 mx-auto mt-16 md:mt-24")
    
    name = player_image.get("alt")
    imgurl = player_image.get("src")

    name_lst.append(name)
    url_lst.append(imgurl)

    print(ind, name)

0 Precious Achiuwa Headshot
1 Jaylen Adams Headshot
2 Steven Adams Headshot
3 Bam Adebayo Headshot
4 LaMarcus Aldridge Headshot
5 Ty-Shon Alexander Headshot
6 Nickeil Alexander-Walker Headshot
7 Grayson Allen Headshot
8 Jarrett Allen Headshot
9 Al-Farouq Aminu Headshot
10 Kyle Anderson Headshot
11 Giannis Antetokounmpo Headshot
12 Kostas Antetokounmpo Headshot
13 Thanasis Antetokounmpo Headshot
14 Carmelo Anthony Headshot
15 Cole Anthony Headshot
16 OG Anunoby Headshot
17 Ryan Arcidiacono Headshot
18 Trevor Ariza Headshot
19 D.J. Augustin Headshot
20 Deni Avdija Headshot
21 Deandre Ayton Headshot
22 Udoka Azubuike Headshot
23 Dwayne Bacon Headshot
24 Marvin Bagley III Headshot
25 LaMelo Ball Headshot
26 Lonzo Ball Headshot
27 Mo Bamba Headshot
28 Desmond Bane Headshot
29 Harrison Barnes Headshot
30 RJ Barrett Headshot
31 Will Barton Headshot
32 Keita Bates-Diop Headshot
33 Nicolas Batum Headshot
34 Aron Baynes Headshot
35 Kent Bazemore Headshot
36 Darius Bazley Headshot
37 Bradley Beal

311 Theo Maledon Headshot
312 Karim Mane Headshot
313 Terance Mann Headshot
314 Nico Mannion Headshot
315 Boban Marjanovic Headshot
316 Lauri Markkanen Headshot
317 Naji Marshall Headshot
318 Caleb Martin Headshot
319 Cody Martin Headshot
320 Jeremiah Martin Headshot
321 Kelan Martin Headshot
322 Kenyon Martin Jr. Headshot
323 Frank Mason Headshot
324 Garrison Mathews Headshot
325 Dakota Mathias Headshot
326 Wesley Matthews Headshot
327 Tyrese Maxey Headshot
328 Skylar Mays Headshot
329 Patrick McCaw Headshot
330 CJ McCollum Headshot
331 T.J. McConnell Headshot
332 Jaden McDaniels Headshot
333 Jalen McDaniels Headshot
334 Doug McDermott Headshot
335 Sean McDermott Headshot
336 JaVale McGee Headshot
337 Rodney McGruder Headshot
338 Alfonzo McKinnie Headshot
339 Jordan McLaughlin Headshot
340 Ben McLemore Headshot
341 Nicolo Melli Headshot
342 De'Anthony Melton Headshot
343 Sam Merrill Headshot
344 Chimezie Metu Headshot
345 Khris Middleton Headshot
346 Darius Miller Headshot
347 Patty M

In [31]:
# download player normal picture
profile_df = pd.DataFrame(data=[name_lst, url_lst]).T
profile_df.columns = ["player", "profile_url"]
profile_df = profile_df.drop_duplicates()

for row in profile_df.iloc[436:].iterrows():
    response = requests.get(row[1]["profile_url"])
    if response.status_code == 200:
        save_name = "{}_{}".format(row[0], row[1]["player"])
        print(save_name)
        with open("nba_player_profile/{}.jpg".format(save_name), "wb") as f:
            f.write(response.content)